# Load Data and Create Charts

In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import mplfinance as mpf
import datetime
import numpy as np
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

- Load data from multiple options chains and create large set

In [2]:
# df_apple = pd.read_csv('data/aapl_2016_2020.csv')
# df_apple['equity'] = 'AAPL'
# df_apple2 = pd.read_csv('data/aapl_2021_2023.csv')
# df_apple2['equity'] = 'AAPL'
# df_tesla = pd.read_csv('data/tsla_2019_2022.csv')
# df_tesla['equity'] = 'TSLA'
# df_spy = pd.read_csv('data/spy_2020_2022.csv')
# df_spy['equity'] = 'SPY'
# df_nividia = pd.read_csv('data/nvda_2020_2022.csv')
# df_nividia['equity'] = 'NVDA'
# df_qqq = pd.read_csv('data/qqq_2020_2022.csv')
# df_qqq['equity'] = 'QQQ'

# df = pd.concat([df_apple,
# df_apple2,
# df_tesla,
# df_spy,
# df_nividia,
# df_qqq])

#df.to_csv('./FullOptions.csv')

- Load CSV

In [3]:
df = pd.read_csv('FullOptions.csv')

In [12]:
pd.set_option('display.max_columns', None) 
df.columns = [i.replace('[','').replace(']','').replace(' ','') for i in df.columns]

df['QUOTE_DATE'] = pd.to_datetime(df['QUOTE_DATE'])


- Convert select columns to float

In [13]:
col_float = list(df.columns[8:-1])
for i in col_float:
    df[i] = pd.to_numeric(df[i], errors='coerce')

- Find unique options

In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
df.sort_values(by='QUOTE_DATE', inplace=True)
df['unique_id'] = [i+'-'+str(j)+'-'+str(k) for i,j,k in zip(df.EXPIRE_DATE,df.STRIKE, df.equity)]
unique_options = df.unique_id.value_counts().index

# Main Graph function

In [15]:
import matplotlib.pyplot as plt
import os
import PIL

def generate_plot(temp, name, quotedate):
    fig, ax1 = plt.subplots(figsize=(.8311688311688312*2, .8311688311688312*2))

    ax1.set_xlabel('')
    ax1.plot(temp['QUOTE_DATE'], temp['C_BID'], linewidth=2, color='black')

    # Hide x-axis ticks and labels
    ax1.set_xticks([])
    ax1.set_xticklabels([])

    # Hide y-axis ticks and labels
    ax1.set_yticks([])
    ax1.set_yticklabels([])

    # Hide the spines (axes)
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.spines['bottom'].set_visible(False)
    ax1.spines['left'].set_visible(False)

    output_folder = './imgs/10p5-train'
    filename = f'option{name}-{quotedate}.png'
    filepath = os.path.join(output_folder, filename)
    plt.savefig(filepath, bbox_inches='tight', pad_inches=0)
    
    img = Image.open(filepath).convert('L')

    # Save the grayscale image
    filepath_grayscale = os.path.join(output_folder, f'option{name}-{quotedate}.png')
    img.save(filepath_grayscale)


# Main Loop --> Randomly sample from dataframe 10K options and create charts and target labels

In [4]:
target_return = []
target_labels = []
target_quote_start_date = []
target_description = []

temp_df = df.iloc[0:10000000,:]
unique_options = list(temp_df.unique_id.value_counts().index)
unique_options = sorted(unique_options, key=lambda x: random.random())

for i in tqdm(unique_options[0:10000]):
    indicator=0
    temp = temp_df.loc[df['unique_id'] == i]
    leny = len(temp) // 15
    for idct in range(leny):
        try:
            i_input = temp.iloc[indicator:indicator + 10, :]
            i_target = temp.iloc[indicator+9:indicator+14, :]

            i_input.fillna(0)

            ret = (i_target['C_BID'].values[-1] - i_target['C_BID'].values[0]) / i_target['C_BID'].values[0]
            target_return.append(ret)
            target_description.append(i)
            target_quote_start_date.append(i_input['QUOTE_DATE'].values[0])

            if i_target['C_BID'].values[0] < i_target['C_BID'].values[-1]:
                target_labels.append(1)
            else:
                target_labels.append(0)

            generate_plot(i_input, i, i_input.QUOTE_DATE.values[0])

            indicator += 15

        except:
            print(f'Couldnt Render {i}')
            break
            
            
target_df = pd.DataFrame({'Label':target_labels, 'Return': target_return, 'Description': target_description, 'Quote_Start_Date': target_quote_start_date})

target_df.to_csv('Target.csv')

print('All Done')

NameError: name 'df' is not defined